<a href="https://colab.research.google.com/github/VanThiKimKhanh1203/Hoc_tang_cuong_2024/blob/main/bai2_giuaky_VTKKhanh.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import random
import pandas as pd

In [2]:
# Hàm chuyển trạng thái
def move(state, action):
    x, y = state
    if action == 'Up' and x > 0:
        return x - 1, y
    elif action == 'Down' and x < M - 1:
        return x + 1, y
    elif action == 'Left' and y > 0:
        return x, y - 1
    elif action == 'Right' and y < N - 1:
        return x, y + 1
    return state

# Hàm chọn hành động với độ chính xác 80%
def take_action(action):
    if random.uniform(0, 1) < 0.8:
        return action
    else:
        return random.choice([a for a in actions if a != action])


In [3]:
# Hàm liệt kê tất cả các đường đi khả thi
def find_all_paths(state, path, visited, target):
    if state == target:
        all_paths.append(path[:])
        return
    x, y = state
    visited[x][y] = True

    for action in actions:
        next_state = move(state, action)
        nx, ny = next_state
        if not visited[nx][ny] and rewards[nx, ny] != -1:
            path.append((state, action, next_state))
            find_all_paths(next_state, path, visited, target)
            path.pop()

    visited[x][y] = False

In [4]:
# Nhập dữ liệu từ người dùng
M = int(input("Nhập số hàng M: "))
N = int(input("Nhập số cột N: "))
num_penalty = int(input("Số ô có điểm -1: "))
penalty_cells = []
for _ in range(num_penalty):
    x, y = map(int, input("Nhập tọa độ ô có điểm -1 (dòng, cột): ").split())
    penalty_cells.append((x - 1, y - 1))

goal = tuple(map(int, input("Nhập tọa độ ô có điểm +10 (dòng, cột): ").split()))
goal = (goal[0] - 1, goal[1] - 1)
sub_goal = tuple(map(int, input("Nhập tọa độ ô có điểm +6 (dòng, cột): ").split()))
sub_goal = (sub_goal[0] - 1, sub_goal[1] - 1)
start = tuple(map(int, input("Nhập tọa độ xuất phát của robot (dòng, cột): ").split()))
start = (start[0] - 1, start[1] - 1)

Nhập số hàng M: 3
Nhập số cột N: 4
Số ô có điểm -1: 1
Nhập tọa độ ô có điểm -1 (dòng, cột): 2 2
Nhập tọa độ ô có điểm +10 (dòng, cột): 2 4
Nhập tọa độ ô có điểm +6 (dòng, cột): 2 3
Nhập tọa độ xuất phát của robot (dòng, cột): 3 1


In [5]:
# Thiết lập các thông số của môi trường
actions = ['Up', 'Down', 'Left', 'Right']
rewards = np.zeros((M, N))
rewards[goal] = 10
rewards[sub_goal] = 6
for cell in penalty_cells:
    rewards[cell] = -1

# Thông số Q-learning
Q = np.zeros((M, N, len(actions)))
gamma = 0.9  # Hệ số giảm phần thưởng
alpha = 0.1  # Tốc độ học
epsilon = 0.1  # Tham số khám phá

# Q-learning
for episode in range(1000):
    state = start
    while state != goal:
        # Chọn hành động với epsilon-greedy
        if random.uniform(0, 1) < epsilon:
            action = random.choice(actions)
        else:
            action = actions[np.argmax(Q[state[0], state[1]])]
        # Áp dụng độ chính xác 80%
        action = take_action(action)
        # Tính trạng thái kế tiếp
        next_state = move(state, action)
        reward = rewards[next_state]
        # Cập nhật Q-value
        Q[state[0], state[1], actions.index(action)] += alpha * (
            reward + gamma * np.max(Q[next_state[0], next_state[1]]) - Q[state[0], state[1], actions.index(action)]
        )
        state = next_state


In [6]:
# Tìm và hiển thị đường đi tối ưu đến sub_goal (+6)
state = start
optimal_path_to_sub_goal = []
while state != sub_goal:
    action = actions[np.argmax(Q[state[0], state[1]])]
    action = take_action(action)
    next_state = move(state, action)
    optimal_path_to_sub_goal.append((state, action, next_state))
    state = next_state

print("\nĐường đi tối ưu đến sub_goal (+6):")
optimal_path_to_sub_goal_str = " ; ".join(f"({step[0][0]}, {step[0][1]}) {step[1]} ({step[2][0]}, {step[2][1]})" for step in optimal_path_to_sub_goal)
print(f"• {optimal_path_to_sub_goal_str}")
print(f"• {rewards[sub_goal]}")


Đường đi tối ưu đến sub_goal (+6):
• (2, 0) Left (2, 0) ; (2, 0) Right (2, 1) ; (2, 1) Right (2, 2) ; (2, 2) Up (1, 2)
• 6.0


In [14]:
# Tìm và hiển thị đường đi tối ưu đến goal (+10)
state = sub_goal
optimal_path_to_goal = []
while state != goal:
    action = actions[np.argmax(Q[state[0], state[1]])]
    action = take_action(action)
    next_state = move(state, action)
    optimal_path_to_goal.append((state, action, next_state))
    state = next_state

print("\nĐường đi tối ưu đến goal (+10):")
optimal_path_to_goal_str = " ; ".join(f"({step[0][0]}, {step[0][1]}) {step[1]} ({step[2][0]}, {step[2][1]})" for step in optimal_path_to_goal)
print(f"• {optimal_path_to_goal_str}")
print(f"• {rewards[goal]}")


Đường đi tối ưu đến goal (+10):
• (1, 2) Up (0, 2) ; (0, 2) Down (1, 2) ; (1, 2) Up (0, 2) ; (0, 2) Down (1, 2) ; (1, 2) Down (2, 2) ; (2, 2) Up (1, 2) ; (1, 2) Up (0, 2) ; (0, 2) Down (1, 2) ; (1, 2) Right (1, 3)
• 10.0


In [15]:
# Hiển thị bảng hướng di chuyển
action_symbols = {'Up': '^', 'Down': 'v', 'Left': '<', 'Right': '>'}
directions_to_sub_goal = np.full((M, N), '', dtype=object)
directions_to_goal = np.full((M, N), '', dtype=object)

for i in range(M):
    for j in range(N):
        if (i, j) == goal:
            directions_to_goal[i, j] = '+10'
        elif (i, j) == sub_goal:
            directions_to_sub_goal[i, j] = '+6'
        elif (i, j) in penalty_cells:
            directions_to_goal[i, j] = directions_to_sub_goal[i, j] = '-1'
        elif (i, j) == start:
            directions_to_goal[i, j] = directions_to_sub_goal[i, j] = 'S'
        else:
            # Hướng đến sub_goal
            if rewards[i, j] == 6 or (rewards[sub_goal] and sub_goal != goal):
                best_action = actions[np.argmax(Q[i, j])]
                directions_to_sub_goal[i, j] = action_symbols[best_action]
            # Hướng đến goal
            if rewards[i, j] == 10 or (rewards[goal] and sub_goal != goal):
                best_action = actions[np.argmax(Q[i, j])]
                directions_to_goal[i, j] = action_symbols[best_action]

print("\nBảng hướng di chuyển đến sub_goal (+6):")
print(pd.DataFrame(directions_to_sub_goal))

print("\nBảng hướng di chuyển đến goal (+10):")
print(pd.DataFrame(directions_to_goal))



Bảng hướng di chuyển đến sub_goal (+6):
   0   1   2  3
0  v   >   v  <
1  >  -1  +6   
2  S   >   ^  <

Bảng hướng di chuyển đến goal (+10):
   0   1  2    3
0  v   >  v    <
1  >  -1     +10
2  S   >  ^    <


In [16]:
# Liệt kê tất cả các đường đi khả thi
visited = [[False for _ in range(N)] for _ in range(M)]
all_paths = []

In [17]:
# Liệt kê tất cả các đường đi khả thi
visited = [[False for _ in range(N)] for _ in range(M)]
all_paths = []

# Tìm tất cả các đường đi khả thi đến sub_goal
find_all_paths(start, [], visited, sub_goal)
print("\nTất cả các đường đi khả thi đến sub_goal (+6):")
for idx, path in enumerate(all_paths, 1):
    print(f"Đường đi {idx}:")
    for step in path:
        print(f"  ({step[0][0]}, {step[0][1]}) {step[1]} ({step[2][0]}, {step[2][1]})")


Tất cả các đường đi khả thi đến sub_goal (+6):
Đường đi 1:
  (2, 0) Up (1, 0)
  (1, 0) Up (0, 0)
  (0, 0) Right (0, 1)
  (0, 1) Right (0, 2)
  (0, 2) Down (1, 2)
Đường đi 2:
  (2, 0) Up (1, 0)
  (1, 0) Up (0, 0)
  (0, 0) Right (0, 1)
  (0, 1) Right (0, 2)
  (0, 2) Right (0, 3)
  (0, 3) Down (1, 3)
  (1, 3) Down (2, 3)
  (2, 3) Left (2, 2)
  (2, 2) Up (1, 2)
Đường đi 3:
  (2, 0) Up (1, 0)
  (1, 0) Up (0, 0)
  (0, 0) Right (0, 1)
  (0, 1) Right (0, 2)
  (0, 2) Right (0, 3)
  (0, 3) Down (1, 3)
  (1, 3) Left (1, 2)
Đường đi 4:
  (2, 0) Right (2, 1)
  (2, 1) Right (2, 2)
  (2, 2) Up (1, 2)
Đường đi 5:
  (2, 0) Right (2, 1)
  (2, 1) Right (2, 2)
  (2, 2) Right (2, 3)
  (2, 3) Up (1, 3)
  (1, 3) Up (0, 3)
  (0, 3) Left (0, 2)
  (0, 2) Down (1, 2)
Đường đi 6:
  (2, 0) Right (2, 1)
  (2, 1) Right (2, 2)
  (2, 2) Right (2, 3)
  (2, 3) Up (1, 3)
  (1, 3) Left (1, 2)


In [18]:
# Tìm tất cả các đường đi khả thi đến goal
all_paths = []
find_all_paths(sub_goal, [], visited, goal)
print("\nTất cả các đường đi khả thi đến goal (+10):")
for idx, path in enumerate(all_paths, 1):
    print(f"Đường đi {idx}:")
    for step in path:
        print(f"  ({step[0][0]}, {step[0][1]}) {step[1]} ({step[2][0]}, {step[2][1]})")


Tất cả các đường đi khả thi đến goal (+10):
Đường đi 1:
  (1, 2) Up (0, 2)
  (0, 2) Left (0, 1)
  (0, 1) Left (0, 0)
  (0, 0) Down (1, 0)
  (1, 0) Down (2, 0)
  (2, 0) Right (2, 1)
  (2, 1) Right (2, 2)
  (2, 2) Right (2, 3)
  (2, 3) Up (1, 3)
Đường đi 2:
  (1, 2) Up (0, 2)
  (0, 2) Right (0, 3)
  (0, 3) Down (1, 3)
Đường đi 3:
  (1, 2) Down (2, 2)
  (2, 2) Left (2, 1)
  (2, 1) Left (2, 0)
  (2, 0) Up (1, 0)
  (1, 0) Up (0, 0)
  (0, 0) Right (0, 1)
  (0, 1) Right (0, 2)
  (0, 2) Right (0, 3)
  (0, 3) Down (1, 3)
Đường đi 4:
  (1, 2) Down (2, 2)
  (2, 2) Right (2, 3)
  (2, 3) Up (1, 3)
Đường đi 5:
  (1, 2) Right (1, 3)
